In [1]:
!pip install onnx onnxruntime onnxsim

     ---------------------------------------- 11.5/11.5 MB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 5.9/5.9 MB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 6.1 MB/s eta 0:00:00
     ---------------------------------------- 6.5/6.5 MB 5.2 MB/s eta 0:00:00
     ---------------------------------------- 46.0/46.0 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 237.5/237.5 kB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 86.8/86.8 kB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 51.1/51.1 kB 2.6 MB/s eta 0:00:00
     -------------------------------------- 532.6/532.6 kB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 95.2/95.2 kB 5.7 MB/s eta 0:00:00


In [2]:
!pip install onnx-tf

     -------------------------------------- 226.1/226.1 kB 2.8 MB/s eta 0:00:00
     -------------------------------------- 742.5/742.5 kB 4.3 MB/s eta 0:00:00


In [2]:
import os
import random

In [5]:
img_dir = 'data/images/train'
images = os.listdir(img_dir)
random.shuffle(images)

In [3]:
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...


In [6]:
%cd yolov7

C:\Users\NDA\PycharmProjects\detect people\yolov7


In [2]:
!python export.py --weights tiny_100_640.pt --grid --end2end --simplify \
--topk-all 100 --iou-thres 0.65 --conf-thres 0.35 --img-size 640 640 --max-wh 640

Import onnx_graphsurgeon failure: No module named 'onnx_graphsurgeon'
Namespace(weights='tiny_100_640.pt', img_size=[640, 640], batch_size=1, dynamic=False, dynamic_batch=False, grid=True, end2end=True, max_wh=640, topk_all=100, iou_thres=0.65, conf_thres=0.35, device='cpu', simplify=True, include_nms=False, fp16=False, int8=False)
Fusing layers... 
IDetect.fuse

Starting TorchScript export with torch 1.12.1+cpu...
TorchScript export success, saved as tiny_100_640.torchscript.pt
CoreML export failure: No module named 'coremltools'

Starting TorchScript-Lite export with torch 1.12.1+cpu...
TorchScript-Lite export success, saved as tiny_100_640.torchscript.ptl

Starting ONNX export with onnx 1.12.0...
onnxruntime

Starting to simplify ONNX...
ONNX export success, saved as tiny_100_640.onnx

Export complete (11.18s). Visualize with https://github.com/lutzroeder/netron.


YOLOR  2022-12-11 torch 1.12.1+cpu CPU

C:\Users\NDA\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 208 layers, 6007596 parameters, 0 gradients, 13.0 GFLOPS
C:\Users\NDA\PycharmProjects\detect people\yolov7\models\yolo.py:150: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4]:
C:\Users\NDA\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\onnx\symbolic_opset9.py:

In [3]:
!onnx-tf convert -i tiny_100_640.onnx -o /yolo7/

2022-12-11 18:42:07,059 - onnx-tf - INFO - Start converting onnx pb to tf saved model
2022-12-11 18:42:07.173761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
2022-12-11 18:42:19,189 - onnx-tf - INFO - Converting completes successfully.
INFO:onnx-tf:Converting completes successfully.


In [7]:
import tensorflow as tf

Convert to tflite float32

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/yolo7/')
tflite_model = converter.convert()
with open('yolov7_model_16.tflite', 'wb') as f:
    f.write(tflite_model)

Convert to tflite float16

In [10]:
converter = tf.lite.TFLiteConverter.from_saved_model('/yolo7/')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()
with open('yolov7_model_99.tflite', 'wb') as f:
    f.write(tflite_model)

In [8]:
def rep_data_gen():
    a = []
    for i in range(150):
        file_name = images[i]
        img = cv2.imread(img_dir + file_name)
        img = cv2.resize(img, (640, 640))
        img = img / 255.0
        img = img.astype(np.float32)
        a.append(img)
    a = np.array(a)
    print(a.shape) # a is np array of 150 3D images
    img = tf.data.Dataset.from_tensor_slices(a).batch(1)
    for i in img.take(BATCH_SIZE):
        print(i)
        yield [i]

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("/yolo7/")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = [tf.uint8]
converter.inference_output_type = [tf.uint8]
converter.representative_dataset=rep_data_gen
tflite_quant_model = converter.convert()